In [ ]:
# ------------------------------------------------------------------------
#
# TITLE - make_iso.ipynb
# AUTHOR - James Lane
# PROJECT - ges-mass
#
# ------------------------------------------------------------------------
#
# Docstrings and metadata:
'''Do some processing on the raw PARSEC v1.2 isochrones to make an isochrone
grid which can be used with the mass fitting program. Specifically, add 
weights which represent the fractional number density of each isochrone point.
'''

__author__ = "James Lane"

In [ ]:
### Imports

## Basic
import numpy as np
import sys, os, pdb, copy, warnings
from matplotlib import pyplot as plt
import scipy.integrate
from tqdm.notebook import tqdm
from isodist import Z2FEH,FEH2Z

# Project specific
sys.path.insert(0,'../../src/')
from ges_mass import util as putil
from ges_mass import iso as piso

### Notebook setup

%matplotlib inline
plt.style.use('../../src/mpl/project.mplstyle') # This must be exactly here
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

### Preliminaries
- The `M_MIN` keyword will be used here to act as a starting point for the isochrone weights calculation.
- Consider changing which IMF is used to calculate the default weights for the code (below). By default it's Chabrier (2003)

In [ ]:
# Keywords
cdict = putil.load_config_to_dict()
keywords = ['BASE_DIR','APOGEE_DR','APOGEE_RESULTS_VERS','GAIA_DR','LOGG_MIN',
            'LOGG_MAX','FEH_MIN','FEH_MAX','FEH_MIN_GSE','FEH_MAX_GSE','M_MIN']
base_dir,apogee_dr,apogee_results_vers,gaia_dr,logg_min,logg_max,feh_min,\
    feh_max,feh_min_gse,feh_max_gse,m_min \
    = putil.parse_config_dict(cdict,keywords)
feh_range_gse = [feh_min_gse,feh_max_gse]
feh_range_all = [feh_min,feh_max]
logg_range = [logg_min,logg_max]
data_dir = base_dir+'data/'
version_dir = 'apogee_'+apogee_dr+'_'+apogee_results_vers+'_gaia_'+gaia_dr+'/'
ga_dir = data_dir+'gaia_apogee/'+version_dir
fig_dir = './fig/'
os.makedirs(fig_dir, exist_ok=True)

### Load data

In [ ]:
# Path for isochrone, I keep it where data for isodist lives
iso_dir = os.environ['ISODIST_DATA']+'/parsec1.2/'
# Two isochrones, one for low metallicity and one for higher metallicity.
# Done separately because of CMD 3.6 limitations, now joined.
iso_names = ['2mass-spitzer-wise-0.0001-z-0.0030-1e10-age-1.4e10.dat',
             '2mass-spitzer-wise-0.0031-z-0.0060-1e10-age-1.4e10.dat']
iso_1 = np.genfromtxt(iso_dir+iso_names[0], names=True, skip_header=13, 
                      comments='#')
iso_2 = np.genfromtxt(iso_dir+iso_names[1], names=True, skip_header=13, 
                      comments='#')
iso = np.concatenate((iso_1,iso_2))

### Normalizations

In [ ]:
# Lambda functions for IMF*mass calculation
chabrier01_imf_mass = lambda mass: piso.chabrier01_lognormal(mass)*mass
chabrier03_imf_mass = lambda mass: piso.chabrier03_lognormal(mass)*mass
kroupa_imf_mass = lambda mass: piso.kroupa(mass)*mass

In [ ]:
imf_norm_chabrier01 = scipy.integrate.quad(chabrier01_imf_mass,0.,100.)[0]
imf_norm_chabrier03 = scipy.integrate.quad(chabrier03_imf_mass,0.,100.)[0]
imf_norm_kroupa = scipy.integrate.quad(kroupa_imf_mass,0.,100.)[0]

### Plot IMFs
Show the Chabrier 2001, Chabrier 2003, Kroupa IMFs

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)

ms = np.logspace(-2,0.,num=100)

ax.plot(ms, piso.chabrier01_lognormal(ms)/imf_norm_chabrier01, 
        color='Red', label='Chabrier 2001')
ax.plot(ms, piso.chabrier03_lognormal(ms)/imf_norm_chabrier03, 
        color='ForestGreen', label='Chabrier 2003')
ax.plot(ms, piso.kroupa(ms)/imf_norm_kroupa, 
        color='DodgerBlue', label='Kroupa')

ax.set_xlabel('m [solar]')
ax.set_ylabel(r'$\xi$(m)')
ax.set_xscale('log')
ax.set_yscale('log')
ax.legend()

fig.savefig(fig_dir+'imfs.png')
fig.show()

### Process Isochrone
First calculate the IMF-based weights and include them in the isochrone. The 
IMF for each isochrone element is the delta of iso['int_IMF'], calculated 
individually for each isochrone of a unique Z and Age. For the first mass point 
I integrate a normalized (1 M solar total) Chabrier (2001) IMF from 0.08 to the 
first initial mass point.

Additionally remove any points with luminosity -9.999, which are WDs that can 
significantly warp the inferred mass fraction

In [ ]:
iso_new = copy.deepcopy(iso)

# Weights from Chabrier (2001) IMF
print('Calculating weights using Chabrier (2001) IMF')
iso_new = piso.calculate_weights_imf(iso_new, norm=imf_norm_chabrier01,
    imf=piso.chabrier01_lognormal, m_min=m_min, 
    weights_key='weights_imf_chabrier01_mmin_'+str(m_min))

# Weights from Chabrier (2003) IMF
print('Calculating weights using Chabrier (2003) IMF')
iso_new = piso.calculate_weights_imf(iso_new, norm=imf_norm_chabrier03,
    imf=piso.chabrier03_lognormal, m_min=m_min, 
    weights_key='weights_imf_chabrier03_mmin_'+str(m_min))

# Weights from Kroupa IMF
print('Calculating weights using Kroupa IMF')
iso_new = piso.calculate_weights_imf(iso_new, norm=imf_norm_kroupa,
    imf=piso.kroupa, m_min=m_min, weights_key='weights_imf_kroupa_mmin_'+str(m_min))

# Weights from differencing 'int_IMF' field
print('Calculating weights by differencing "int_IMF" field in the isochrone')
iso_new = piso.calculate_weights_imf(iso_new, weights_key='weights_imf_diff',
                                        diff=True, diff_key='int_IMF')

# Make one set of weights that will be the default: 'weights_imf'
w_imf = np.zeros(len(iso), dtype=[('weights_imf','f8'),])
w_imf['weights_imf'] = iso_new['weights_imf_chabrier03_mmin_'+str(m_min)]
iso_new = np.lib.recfunctions.merge_arrays((iso_new,w_imf), flatten=True)

iso_new = iso_new[iso_new['logL'] > -9]

In [ ]:
# Save, store it with all of the other things required to do density modelling
np.save(ga_dir+'iso_grid.npy',iso_new)

### Calculate some Isochrone factors, Plot weights

In [ ]:
# Assuming a standard set of minimum (J-K) determine the 
# average mass in giants and mass ratio factors
jkmins = [0.3,0.5]

def determine_isofactors(iso,feh_range,logg_range,jkmin,weights_key):
    # The mass ratio mask is for all stars considered, determined by 
    # [Fe/H] and age
    massratio_isomask = (Z2FEH(iso['Zini']) > feh_range[0]) &\
                        (Z2FEH(iso['Zini']) < feh_range[1]) &\
                        (iso['logAge'] >= 10) &\
                        (iso['logL'] > -9) # Eliminates WDs
    # The average mass mask extracts fitted sample based on color and logg
    # but also cuts on [Fe/H] and age (from massratio_isomask)
    avmass_isomask = massratio_isomask &\
                     (iso['Jmag']-iso['Ksmag'] > jkmin) &\
                     (iso['logg'] > logg_range[0]) &\
                     (iso['logg'] < logg_range[1])
    massratio = piso.mass_ratio(iso[massratio_isomask], logg_range=logg_range,
                                jk_range=[jkmin,999.], weights_key=weights_key)
    avmass = piso.average_mass(iso[avmass_isomask], weights_key=weights_key)
    return avmass, massratio

weights_keys = ['weights_imf_chabrier01_mmin_'+str(m_min),
                'weights_imf_chabrier03_mmin_'+str(m_min),
                'weights_imf_kroupa_mmin_'+str(m_min),
                'weights_imf_diff']

for i in range(len(jkmins)):
    for j in range(2):
        if j==0:
            feh_range = feh_range_all
        else:
            feh_range = feh_range_gse
        for k in range(len(weights_keys)):
    
            avmass, massratio = determine_isofactors(iso_new, feh_range, 
                                                     logg_range, jkmin=jkmins[i], 
                                                     weights_key=weights_keys[k])
            print('\nFor minimum (J-K)='+str(jkmins[i])+', Fe/H range: '+\
                  str(feh_range)+', weights_key='+str(weights_keys[k]))
            print('Average mass of giants: '+str(round(avmass,3)))
            print('Mass ratio of giants to whole isochrone: '+str(round(massratio,5)))
            print('Isochrone factor: '+str(round(avmass/massratio,1)))
        print('\n----------')           

In [ ]:
# Unique metallicity and ages
unique_zini = np.unique(iso['Zini'])
unique_age = np.unique(iso['logAge'])
jkmin = 0.5

for i in range(len(unique_zini)):
    for j in range(len(unique_age)):
        if i%10 != 0: continue
        iso_mask = np.where((iso_new['Zini']==unique_zini[i]) &\
                            (iso_new['logAge']==unique_age[j]))[0]
        this_iso = iso_new[iso_mask]
        
        fig = plt.figure()
        ax = fig.add_subplot(111)
        
        ax.plot(iso_new[iso_mask][1:]['Mini'], 
                iso_new[iso_mask][1:]['weights_imf_chabrier01_mmin_'+str(m_min)],
                color='Red', label='Chabrier 2001', zorder=1)
        ax.plot(iso_new[iso_mask][1:]['Mini'], 
                iso_new[iso_mask][1:]['weights_imf_chabrier03_mmin_'+str(m_min)],
                color='Orange', label='Chabrier 2003', zorder=1)
        ax.plot(iso_new[iso_mask][1:]['Mini'], 
                iso_new[iso_mask][1:]['weights_imf_kroupa_mmin_'+str(m_min)],
                color='ForestGreen', label='Kroupa', zorder=1)
        ax.plot(iso_new[iso_mask][1:]['Mini'], 
                iso_new[iso_mask][1:]['weights_imf_diff'],
                color='DodgerBlue', linestyle='dashed', zorder=2, label='Diff')
        ax.set_xlabel(r'M$_{ini}$')
        ax.set_ylabel(r'w$_{i}$')
        ax.legend()
        fig.savefig(fig_dir+'z'+str(unique_zini[i])+'_logAge'+\
                    str(unique_age[j])+'_weights_no_m0.png', dpi=100)
        plt.close()